<a href="https://colab.research.google.com/github/S1LV4/DPECalculadoraHipo/blob/main/calc_hipo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Calculadora de Hipossuficiência Econômica - Defensoria Pública

Este projeto foi desenvolvido como parte do Processo Seletivo nº 40 – CATE/RS – Defensoria Pública, e implementa os critérios estabelecidos na Resolução nº 07/2018 do Conselho Superior da Defensoria Pública para aferição da hipossuficiência econômica de assistidos(as), a fim de verificar o direito ao atendimento jurídico gratuito.

## Regras de Cálculo:

1. **Renda Bruta Familiar**: Soma de todos os valores declarados
2. **Descontos por Dependentes**:
   - Cada dependente ordinário: reduz 25% do SMN
   - Cada dependente especial: reduz 50% do SMN
3. **Critério de Hipossuficiência**: Renda ajustada ≤ 3 SMN

In [ ]:
import requests
from datetime import datetime
import json

In [ ]:
def obter_salario_minimo_atual():
    """
    Obtém o salário mínimo nacional atual da API do BACEN.
    Em caso de falha, retorna um valor padrão atualizado.
    """
    try:
        # Tentativa de obter de API pública (exemplo)
        # A API do Banco Central do Brasil fornece o valor do salário mínimo / Ultimo salário mínimo
        response = requests.get('https://api.bcb.gov.br/dados/serie/bcdata.sgs.1619/dados/ultimos/1?formato=json', timeout=5)
        if response.status_code == 200:
            data = response.json()
            return float(data[0]['valor'])
    except:
        pass

    # Valor padrão (atualizar conforme legislação)
    return 1518.00  # Salário mínimo de 2025


In [ ]:
def calcular_hipossuficiencia(renda_bruta_familiar, dependentes_ordinarios, dependentes_especiais, salario_minimo=None):
    """
    Calcula se o assistido atende aos critérios de hipossuficiência econômica.

    Parâmetros:
    - renda_bruta_familiar (float): Renda bruta total da família em reais
    - dependentes_ordinarios (int): Quantidade de dependentes ordinários
    - dependentes_especiais (int): Quantidade de dependentes especiais
    - salario_minimo (float, opcional): Valor do salário mínimo. Se None, obtém automaticamente

    Retorna:
    - dict: Resultado detalhado do cálculo
    """

    # Obter salário mínimo se não fornecido
    if salario_minimo is None:
        salario_minimo = obter_salario_minimo_atual()

    # Validações de entrada
    if renda_bruta_familiar < 0:
        raise ValueError("A renda bruta familiar não pode ser negativa")
    if dependentes_ordinarios < 0 or dependentes_especiais < 0:
        raise ValueError("O número de dependentes não pode ser negativo")
    if salario_minimo <= 0:
        raise ValueError("O salário mínimo deve ser maior que zero")

    # Cálculos conforme Resolução 07/2018
    desconto_dependentes_ordinarios = dependentes_ordinarios * (salario_minimo * 0.25)
    desconto_dependentes_especiais = dependentes_especiais * (salario_minimo * 0.50)

    desconto_total_dependentes = desconto_dependentes_ordinarios + desconto_dependentes_especiais
    renda_ajustada = renda_bruta_familiar - desconto_total_dependentes

    # Garantir que a renda ajustada não seja negativa
    renda_ajustada = max(renda_ajustada, 0)

    limite_hipossuficiencia = salario_minimo * 3
    eh_hipossuficiente = renda_ajustada <= limite_hipossuficiencia

    # Preparar resultado detalhado
    resultado = {
        'data_calculo': datetime.now().strftime('%d/%m/%Y %H:%M:%S'),
        'salario_minimo': salario_minimo,
        'renda_bruta_familiar': renda_bruta_familiar,
        'dependentes_ordinarios': dependentes_ordinarios,
        'dependentes_especiais': dependentes_especiais,
        'desconto_dependentes_ordinarios': desconto_dependentes_ordinarios,
        'desconto_dependentes_especiais': desconto_dependentes_especiais,
        'desconto_total_dependentes': desconto_total_dependentes,
        'renda_ajustada': renda_ajustada,
        'limite_hipossuficiencia': limite_hipossuficiencia,
        'eh_hipossuficiente': eh_hipossuficiente,
        'percentual_limite_utilizado': (renda_ajustada / limite_hipossuficiencia) * 100
    }

    return resultado

In [ ]:
def gerar_relatorio_detalhado(resultado):
    """
    Gera um relatório detalhado e justificado do resultado da análise de hipossuficiência.
    """
    r = resultado

    relatorio = f"""
╔═══════════════════════════════════════════════════════════════════════════════════╗
║                    RELATÓRIO DE ANÁLISE DE HIPOSSUFICIÊNCIA                      ║
║                         Defensoria Pública - Resolução 07/2018                   ║
╚═══════════════════════════════════════════════════════════════════════════════════╝

📅 Data do Cálculo: {r['data_calculo']}
💰 Salário Mínimo Nacional: R$ {r['salario_minimo']:,.2f}

┌─────────────────────────────────────────────────────────────────────────────────┐
│                                DADOS INFORMADOS                                 │
├─────────────────────────────────────────────────────────────────────────────────┤
│ Renda Bruta Familiar:       R$ {r['renda_bruta_familiar']:>12,.2f}              │
│ Dependentes Ordinários:     {r['dependentes_ordinarios']:>15} pessoa(s)         │
│ Dependentes Especiais:      {r['dependentes_especiais']:>15} pessoa(s)          │
└─────────────────────────────────────────────────────────────────────────────────┘

┌─────────────────────────────────────────────────────────────────────────────────┐
│                            CÁLCULO DOS DESCONTOS                                │
├─────────────────────────────────────────────────────────────────────────────────┤
│ Ordinários: {r['dependentes_ordinarios']} × (R$ {r['salario_minimo']:,.2f} × 25%) │
│            = R$ {r['desconto_dependentes_ordinarios']:>10,.2f}                                   │
│ Especiais:  {r['dependentes_especiais']} × (R$ {r['salario_minimo']:,.2f} × 50%)  │
│            = R$ {r['desconto_dependentes_especiais']:>10,.2f}                                   │
│                                                                                 │
│ ➤ DESCONTO TOTAL: R$ {r['desconto_total_dependentes']:>12,.2f}                                │
└─────────────────────────────────────────────────────────────────────────────────┘

┌─────────────────────────────────────────────────────────────────────────────────┐
│                              RENDA AJUSTADA                                     │
├─────────────────────────────────────────────────────────────────────────────────┤
│ Renda Bruta:               R$ {r['renda_bruta_familiar']:>12,.2f}               │
│ (-) Descontos:             R$ {r['desconto_total_dependentes']:>12,.2f}         │
│ (=) Renda Ajustada:        R$ {r['renda_ajustada']:>12,.2f}                     │
└─────────────────────────────────────────────────────────────────────────────────┘

┌─────────────────────────────────────────────────────────────────────────────────┐
│                          ANÁLISE DE HIPOSSUFICIÊNCIA                            │
├─────────────────────────────────────────────────────────────────────────────────┤
│ Limite (3 × SMN):         R$ {r['limite_hipossuficiencia']:>12,.2f}             │
│ Renda Ajustada:           R$ {r['renda_ajustada']:>12,.2f}                      │
│ Percentual Utilizado:     {r['percentual_limite_utilizado']:>12.1f}%            │
└─────────────────────────────────────────────────────────────────────────────────┘
"""

    if r['eh_hipossuficiente']:
        relatorio += f"""
╔═══════════════════════════════════════════════════════════════════════════════════╗
║                                   ✅ RESULTADO                                    ║
║                                                                                   ║
║                            ASSISTIDO HIPOSSUFICIENTE                              ║
║                                                                                   ║
║ JUSTIFICATIVA: A renda ajustada de R$ {r['renda_ajustada']:,.2f} está dentro do limite      ║
║ de R$ {r['limite_hipossuficiencia']:,.2f} (equivalente a 3 salários mínimos), conforme a     ║
║ Resolução nº 07/2018 do Conselho Superior da Defensoria Pública.                 ║
║                                                                                   ║
║ O(a) assistido(a) tem direito à assistência jurídica gratuita da Defensoria.     ║
╚═══════════════════════════════════════════════════════════════════════════════════╝
"""
    else:
        relatorio += f"""
╔═══════════════════════════════════════════════════════════════════════════════════╗
║                                   ❌ RESULTADO                                    ║
║                                                                                   ║
║                          ASSISTIDO NÃO HIPOSSUFICIENTE                            ║
║                                                                                   ║
║ JUSTIFICATIVA: A renda ajustada de R$ {r['renda_ajustada']:,.2f} ultrapassa o limite de     ║
║ R$ {r['limite_hipossuficiencia']:,.2f} (equivalente a 3 salários mínimos), não atendendo aos║
║ critérios estabelecidos pela Resolução nº 07/2018 do Conselho Superior.          ║
║                                                                                   ║
║ O(a) assistido(a) não se enquadra, por esse critério, para atendimento gratuito. ║
╚═══════════════════════════════════════════════════════════════════════════════════╝
"""

    return relatorio


## Exemplos de Uso

Vamos testar a calculadora com alguns casos práticos:

In [ ]:
# Exemplo 1: Família com renda baixa e dependentes
print("=" * 80)
print("EXEMPLO 1: Família com 2 dependentes ordinários")
print("=" * 80)

resultado1 = calcular_hipossuficiencia(
    renda_bruta_familiar=2800.00,
    dependentes_ordinarios=5,
    dependentes_especiais=0,
    salario_minimo=1518.00
)

print(gerar_relatorio_detalhado(resultado1))

EXEMPLO 1: Família com 2 dependentes ordinários

╔═══════════════════════════════════════════════════════════════════════════════════╗
║                    RELATÓRIO DE ANÁLISE DE HIPOSSUFICIÊNCIA                      ║
║                         Defensoria Pública - Resolução 07/2018                   ║
╚═══════════════════════════════════════════════════════════════════════════════════╝

📅 Data do Cálculo: 06/08/2025 16:06:30
💰 Salário Mínimo Nacional: R$ 1,518.00

┌─────────────────────────────────────────────────────────────────────────────────┐
│                                DADOS INFORMADOS                                 │
├─────────────────────────────────────────────────────────────────────────────────┤
│ Renda Bruta Familiar:       R$     2,800.00              │
│ Dependentes Ordinários:                   5 pessoa(s)         │
│ Dependentes Especiais:                    0 pessoa(s)          │
└─────────────────────────────────────────────────────────────────────────────────┘

In [ ]:
# Exemplo 2: Família com renda alta
print("=" * 80)
print("EXEMPLO 2: Família com renda acima do limite")
print("=" * 80)

resultado2 = calcular_hipossuficiencia(
    renda_bruta_familiar=5000.00,
    dependentes_ordinarios=1,
    dependentes_especiais=0,
    salario_minimo=1518.00
)

print(gerar_relatorio_detalhado(resultado2))

EXEMPLO 2: Família com renda acima do limite

╔═══════════════════════════════════════════════════════════════════════════════════╗
║                    RELATÓRIO DE ANÁLISE DE HIPOSSUFICIÊNCIA                      ║
║                         Defensoria Pública - Resolução 07/2018                   ║
╚═══════════════════════════════════════════════════════════════════════════════════╝

📅 Data do Cálculo: 06/08/2025 16:06:30
💰 Salário Mínimo Nacional: R$ 1,518.00

┌─────────────────────────────────────────────────────────────────────────────────┐
│                                DADOS INFORMADOS                                 │
├─────────────────────────────────────────────────────────────────────────────────┤
│ Renda Bruta Familiar:       R$     5,000.00              │
│ Dependentes Ordinários:                   1 pessoa(s)         │
│ Dependentes Especiais:                    0 pessoa(s)          │
└─────────────────────────────────────────────────────────────────────────────────┘

┌

In [ ]:
# Exemplo 3: Família com dependente especial
print("=" * 80)
print("EXEMPLO 3: Família com 1 dependente especial")
print("=" * 80)

resultado3 = calcular_hipossuficiencia(
    renda_bruta_familiar=3500.00,
    dependentes_ordinarios=0,
    dependentes_especiais=1,
    salario_minimo=1518.00
)

print(gerar_relatorio_detalhado(resultado3))

EXEMPLO 3: Família com 1 dependente especial

╔═══════════════════════════════════════════════════════════════════════════════════╗
║                    RELATÓRIO DE ANÁLISE DE HIPOSSUFICIÊNCIA                      ║
║                         Defensoria Pública - Resolução 07/2018                   ║
╚═══════════════════════════════════════════════════════════════════════════════════╝

📅 Data do Cálculo: 06/08/2025 16:06:30
💰 Salário Mínimo Nacional: R$ 1,518.00

┌─────────────────────────────────────────────────────────────────────────────────┐
│                                DADOS INFORMADOS                                 │
├─────────────────────────────────────────────────────────────────────────────────┤
│ Renda Bruta Familiar:       R$     3,500.00              │
│ Dependentes Ordinários:                   0 pessoa(s)         │
│ Dependentes Especiais:                    1 pessoa(s)          │
└─────────────────────────────────────────────────────────────────────────────────┘

┌

## Função Interativa para Novos Cálculos

In [ ]:
def calculadora_interativa():
    """
    Versão interativa da calculadora para facilitar o uso.
    """
    print("╔═══════════════════════════════════════════════════════════════╗")
    print("║          CALCULADORA DE HIPOSSUFICIÊNCIA ECONÔMICA           ║")
    print("║               Defensoria Pública - Resolução 07/2018         ║")
    print("╚═══════════════════════════════════════════════════════════════╝\n")

    try:
        # Coleta de dados
        renda_bruta = float(input("💰 Informe a renda bruta familiar (R$): "))
        dep_ord = int(input("👥 Número de dependentes ordinários: "))
        dep_esp = int(input("🔆 Número de dependentes especiais: "))

        usar_smn_auto = input("🤖 Usar salário mínimo atual automático? (s/n): ").lower().startswith('s')

        if usar_smn_auto:
            smn = None
        else:
            smn = float(input("💵 Valor do salário mínimo (R$): "))

        # Cálculo
        resultado = calcular_hipossuficiencia(renda_bruta, dep_ord, dep_esp, smn)

        # Exibição do resultado
        print("\n" + "="*80)
        print("PROCESSANDO...")
        print("="*80)

        print(gerar_relatorio_detalhado(resultado))

        # Opção de salvar
        salvar = input("\n💾 Deseja salvar o resultado em arquivo? (s/n): ").lower().startswith('s')
        if salvar:
            nome_arquivo = f"relatorio_hipossuficiencia_{datetime.now().strftime('%Y%m%d_%H%M%S')}.txt"
            with open(nome_arquivo, 'w', encoding='utf-8') as f:
                f.write(gerar_relatorio_detalhado(resultado))
            print(f"✅ Relatório salvo como: {nome_arquivo}")

    except ValueError as e:
        print(f"❌ Erro nos dados informados: {e}")
    except Exception as e:
        print(f"❌ Erro inesperado: {e}")

# Descomente a linha abaixo para usar a versão interativa
# calculadora_interativa()

╔═══════════════════════════════════════════════════════════════╗
║          CALCULADORA DE HIPOSSUFICIÊNCIA ECONÔMICA           ║
║               Defensoria Pública - Resolução 07/2018         ║
╚═══════════════════════════════════════════════════════════════╝


PROCESSANDO...

╔═══════════════════════════════════════════════════════════════════════════════════╗
║                    RELATÓRIO DE ANÁLISE DE HIPOSSUFICIÊNCIA                      ║
║                         Defensoria Pública - Resolução 07/2018                   ║
╚═══════════════════════════════════════════════════════════════════════════════════╝

📅 Data do Cálculo: 06/08/2025 16:06:42
💰 Salário Mínimo Nacional: R$ 1,518.00

┌─────────────────────────────────────────────────────────────────────────────────┐
│                                DADOS INFORMADOS                                 │
├─────────────────────────────────────────────────────────────────────────────────┤
│ Renda Bruta Familiar:       R$     8,000.00 

## Análise Estatística (Opcional)

Função para análise de múltiplos casos:

In [ ]:
def analise_multiplos_casos(casos):
    """
    Analisa múltiplos casos e gera estatísticas.

    casos: lista de dicionários com as chaves:
           'renda_bruta', 'dep_ordinarios', 'dep_especiais', 'nome' (opcional)
    """
    resultados = []
    hipossuficientes = 0

    print("📊 ANÁLISE DE MÚLTIPLOS CASOS")
    print("=" * 60)

    for i, caso in enumerate(casos, 1):
        nome = caso.get('nome', f'Caso {i}')
        resultado = calcular_hipossuficiencia(
            caso['renda_bruta'],
            caso['dep_ordinarios'],
            caso['dep_especiais']
        )
        resultado['nome'] = nome
        resultados.append(resultado)

        if resultado['eh_hipossuficiente']:
            hipossuficientes += 1
            status = "✅ HIPOSSUFICIENTE"
        else:
            status = "❌ NÃO HIPOSSUFICIENTE"

        print(f"{nome}: {status} (Renda Ajustada: R$ {resultado['renda_ajustada']:,.2f})")

    # Estatísticas
    total_casos = len(casos)
    percentual_hipossuficientes = (hipossuficientes / total_casos) * 100

    print("\n" + "=" * 60)
    print("📈 ESTATÍSTICAS GERAIS")
    print("=" * 60)
    print(f"Total de casos analisados: {total_casos}")
    print(f"Casos hipossuficientes: {hipossuficientes} ({percentual_hipossuficientes:.1f}%)")
    print(f"Casos não hipossuficientes: {total_casos - hipossuficientes} ({100 - percentual_hipossuficientes:.1f}%)")

    return resultados

# Exemplo de uso da análise múltipla
casos_exemplo = [
    {'nome': 'Família Silva', 'renda_bruta': 2500, 'dep_ordinarios': 2, 'dep_especiais': 0},
    {'nome': 'Família Santos', 'renda_bruta': 4500, 'dep_ordinarios': 1, 'dep_especiais': 1},
    {'nome': 'Família Costa', 'renda_bruta': 1800, 'dep_ordinarios': 0, 'dep_especiais': 1},
    {'nome': 'Família Lima', 'renda_bruta': 5200, 'dep_ordinarios': 0, 'dep_especiais': 0},
]

analise_multiplos_casos(casos_exemplo)

📊 ANÁLISE DE MÚLTIPLOS CASOS
Família Silva: ✅ HIPOSSUFICIENTE (Renda Ajustada: R$ 1,741.00)
Família Santos: ✅ HIPOSSUFICIENTE (Renda Ajustada: R$ 3,361.50)
Família Costa: ✅ HIPOSSUFICIENTE (Renda Ajustada: R$ 1,041.00)
Família Lima: ❌ NÃO HIPOSSUFICIENTE (Renda Ajustada: R$ 5,200.00)

📈 ESTATÍSTICAS GERAIS
Total de casos analisados: 4
Casos hipossuficientes: 3 (75.0%)
Casos não hipossuficientes: 1 (25.0%)


[{'data_calculo': '06/08/2025 16:06:45',
  'salario_minimo': 1518.0,
  'renda_bruta_familiar': 2500,
  'dependentes_ordinarios': 2,
  'dependentes_especiais': 0,
  'desconto_dependentes_ordinarios': 759.0,
  'desconto_dependentes_especiais': 0.0,
  'desconto_total_dependentes': 759.0,
  'renda_ajustada': 1741.0,
  'limite_hipossuficiencia': 4554.0,
  'eh_hipossuficiente': True,
  'percentual_limite_utilizado': 38.23012736056214,
  'nome': 'Família Silva'},
 {'data_calculo': '06/08/2025 16:06:45',
  'salario_minimo': 1518.0,
  'renda_bruta_familiar': 4500,
  'dependentes_ordinarios': 1,
  'dependentes_especiais': 1,
  'desconto_dependentes_ordinarios': 379.5,
  'desconto_dependentes_especiais': 759.0,
  'desconto_total_dependentes': 1138.5,
  'renda_ajustada': 3361.5,
  'limite_hipossuficiencia': 4554.0,
  'eh_hipossuficiente': True,
  'percentual_limite_utilizado': 73.81422924901186,
  'nome': 'Família Santos'},
 {'data_calculo': '06/08/2025 16:06:45',
  'salario_minimo': 1518.0,
  're

## Notas Importantes

1. **Base Legal**: Este código implementa a Resolução 07/2018 do Conselho Superior da Defensoria Pública
2. **Salário Mínimo**: A função tenta obter o valor atual automaticamente, mas pode ser informado manualmente
3. **Validações**: O código inclui validações para garantir a integridade dos dados
4. **Relatórios**: Gera relatórios detalhados e justificados para cada análise
5. **Flexibilidade**: Permite análise individual ou em lote de múltiplos casos

Para usar este notebook:
1. Execute todas as células sequencialmente
2. Use a função `calcular_hipossuficiencia()` para casos individuais
3. Use `calculadora_interativa()` para uma interface mais amigável
4. Use `analise_multiplos_casos()` para análise estatística